In [2]:
from google.colab import files

# Upload multiple files
uploaded = files.upload()



Saving main.csv to main.csv
Saving products.csv to products.csv
Saving reviews.csv to reviews.csv
Saving sales.csv to sales.csv


In [2]:
from google.colab import files
uploaded = files.upload()

import os
print(os.listdir())  # This should now list your uploaded files




Saving main.csv to main.csv
Saving products.csv to products.csv
Saving reviews.csv to reviews.csv
Saving sales.csv to sales.csv
['.config', 'reviews.csv', 'main.csv', 'products.csv', 'sales.csv', 'sample_data']


In [10]:
import pandas as pd

df_main = pd.read_csv("main.csv", encoding="utf-8")
df_reviews = pd.read_csv("reviews.csv", encoding="utf-8")
df_sales = pd.read_csv("sales.csv", encoding="utf-8")
df_products = pd.read_csv("products.csv", encoding="utf-8")

# Display the first few rows of each dataframe
df_main.head(), df_reviews.head(), df_sales.head(), df_products.head()


(                            Uniq Id            Crawl Timestamp  \
 0  51b010b871cde349bd32159a1cc1a15f  2020-01-24 16:08:36 +0000   
 1  d6a7f100e44a626a3701804e99236ad6  2020-01-24 15:54:21 +0000   
 2  99d2b7da7e3e427a942f864937dacd9d  2020-01-24 18:34:28 +0000   
 3  4c76d170c2c6a759cbce812d790a0b88  2020-01-24 11:08:53 +0000   
 4  8ac95837dc8baa01e504fd8f633ffaf2  2020-03-10 07:37:21 +0000   
 
                                          Product Url  \
 0  https://www.walmart.com/ip/Allegiance-Economy-...   
 1  https://www.walmart.com/ip/Kenneth-Cole-Reacti...   
 2  https://www.walmart.com/ip/Kid-Tough-Fitness-I...   
 3  https://www.walmart.com/ip/THE-FIRST-YEARS/167...   
 4  https://www.walmart.com/ip/4-Pack-MD-USA-Seaml...   
 
                                         Product Name  \
 0  Allegiance Economy Dual-scale Digital Thermometer   
 1  Kenneth Cole Reaction Eau De Parfum Spray For ...   
 2  Kid Tough Fitness Inflatable Free-Standing Pun...   
 3                      

In [1]:
!mkdir -p /content/data  # Create a folder to store files

# Move files to the new folder
!mv main.csv /content/data/
!mv reviews.csv /content/data/
!mv sales.csv /content/data/
!mv products.csv /content/data/


mv: cannot stat 'main.csv': No such file or directory
mv: cannot stat 'reviews.csv': No such file or directory
mv: cannot stat 'sales.csv': No such file or directory
mv: cannot stat 'products.csv': No such file or directory


In [2]:
import pandas as pd

df_main = pd.read_csv("/content/data/main.csv", encoding="utf-8")
df_reviews = pd.read_csv("/content/data/reviews.csv", encoding="utf-8")
df_sales = pd.read_csv("/content/data/sales.csv", encoding="utf-8")
df_products = pd.read_csv("/content/data/products.csv", encoding="utf-8")

df_main.head(), df_reviews.head(), df_sales.head(), df_products.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/main.csv'

In [6]:
print("Main CSV Columns:", df_main.columns.tolist())
print("Reviews CSV Columns:", df_reviews.columns.tolist())
print("Sales CSV Columns:", df_sales.columns.tolist())
print("Products CSV Columns:", df_products.columns.tolist())



Main CSV Columns: ['Uniq Id', 'Crawl Timestamp', 'Product Url', 'Product Name', 'Description', 'List Price', 'Sale Price', 'Brand', 'Item Number', 'Gtin', 'Package Size', 'Category', 'Postal Code', 'Available', 'ProductID', 'CustID', 'Label']
Reviews CSV Columns: ['customer_id', 'product_id', 'uniq_id', 'review', 'label', 'name', 'location', 'Date', 'Rating', 'Name', 'Location', 'Image_Links', 'CustID', 'ProductID']
Sales CSV Columns: ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category', 'Purchase', 'ProductID', 'CustID', 'Category', 'Product_Name']
Products CSV Columns: ['Unnamed: 0', 'ProductID', 'ProductName', 'Description', 'Price', 'Category', 'StockDate']


In [7]:
import pandas as pd

# Step 1: Extract category mappings from df_sales (Unique Short Categories)
category_mappings = df_sales[['Category']].drop_duplicates()
category_mappings = category_mappings.dropna().reset_index(drop=True)

# Step 2: Create a dictionary mapping for categories
# We assume `df_main['Category']` has longer format, and we match it with `df_sales['Category']`
category_map = {}
for category in df_main['Category'].dropna().unique():
    matched_category = category_mappings['Category'][category_mappings['Category'].str.contains(category.split('|')[0].strip(), case=False, na=False)]

    if not matched_category.empty:
        category_map[category] = matched_category.values[0]  # Assign the first match found
    else:
        category_map[category] = "Other"  # Default fallback if no match is found

# Step 3: Apply the mapping to df_main
df_main['Category'] = df_main['Category'].map(category_map)

# Step 4: Select relevant columns from df_main
df_main_selected = df_main[['CustID', 'ProductID', 'Uniq Id', 'Crawl Timestamp', 'Product Url', 'Product Name',
                            'Description', 'List Price', 'Sale Price', 'Brand', 'Item Number', 'Gtin', 'Package Size',
                            'Category', 'Postal Code', 'Available', 'Label']]

# Step 5: Select relevant columns from df_reviews
df_reviews_selected = df_reviews[['CustID', 'ProductID', 'Date', 'name', 'location', 'Rating', 'review', 'Image_Links']]

# Step 6: Merge both datasets using INNER JOIN on `CustID` and `ProductID`
df_merged = pd.merge(df_main_selected, df_reviews_selected, on=["CustID", "ProductID"], how="inner")

# Step 7: Fill remaining null values with synthetic examples from existing data
for col in df_merged.columns:
    if df_merged[col].isnull().sum() > 0:
        if df_merged[col].dtype == "object":
            df_merged[col].fillna(df_merged[col].mode()[0], inplace=True)  # Fill with mode for categorical columns
        else:
            df_merged[col].fillna(df_merged[col].median(), inplace=True)  # Fill with median for numerical columns

# Step 8: Save and Download the Merged Data
from google.colab import files

output_path = "merged_data.csv"
df_merged.to_csv(output_path, index=False)
files.download(output_path)

print("✅ Merging Complete! Merged dataset saved at:", output_path)



<ipython-input-7-c8488ba1e0cb>:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged[col].fillna(df_merged[col].mode()[0], inplace=True)  # Fill with mode for categorical columns
<ipython-input-7-c8488ba1e0cb>:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Merging Complete! Merged dataset saved at: merged_data.csv


In [35]:
import pandas as pd

# ✅ Step 1: Extract past purchase history by matching CustID in df_main and df_sales
df_past_purchases = df_sales[df_sales['CustID'].isin(df_main['CustID'])]

# ✅ Step 1.1: Merge with df_main to bring in Brand and Category (Ensuring no duplicates)
df_past_purchases = df_past_purchases.merge(
    df_main[['ProductID', 'Brand', 'Category']],
    on='ProductID',
    how='left'
)

# Rename Category to avoid duplicates if both exist
if 'Category_x' in df_past_purchases.columns and 'Category_y' in df_past_purchases.columns:
    df_past_purchases.drop(columns=['Category_x'], inplace=True)  # Drop one category column
    df_past_purchases.rename(columns={'Category_y': 'Category'}, inplace=True)  # Keep a single category column

# ✅ Step 2: Identify top-selling products using ProductID from df_main, df_reviews, and df_sales
top_selling_sales = df_sales['ProductID'].value_counts().reset_index()
top_selling_sales.columns = ['ProductID', 'Total_Sales']

top_selling_reviews = df_reviews['ProductID'].value_counts().reset_index()
top_selling_reviews.columns = ['ProductID', 'Total_Reviews']

# 🔄 Merge sales and reviews to get an overall popularity score
top_selling_products = pd.merge(top_selling_sales, top_selling_reviews, on='ProductID', how='outer').fillna(0)
top_selling_products['Popularity_Score'] = top_selling_products['Total_Sales'] + top_selling_products['Total_Reviews']
top_selling_products = top_selling_products.sort_values(by='Popularity_Score', ascending=False)

# ✅ Step 3: Filter store-priority products using the Label column (Rollback/Sale)
store_priority_products = df_main[df_main['Label'].isin(['Rollback', 'On Sale'])]

# ✅ Display extracted data
print("✅ Data extraction complete!")

# 🔍 Show first few rows for validation
display(df_past_purchases.head())  # Past purchase history with Brand and Category fixed
display(top_selling_products.head())  # Top-selling products
display(store_priority_products.head())  # Store-priority products


✅ Data extraction complete!


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category,Purchase,ProductID,CustID,Product_Name,Brand,Category
0,1000001,P00069042,F,0-17,10,A,2,0,3,8370,2094,5130,Pro-Oxide Cream Developer 20 Volume By Redken ...,Redken,Premium Beauty
1,1000001,P00248942,F,0-17,10,A,2,0,1,15200,173,5130,Fishing Tackle Waist Bag Reel Lure Gear Storag...,Generic,Sports & Outdoors
2,1000001,P00087842,F,0-17,10,A,2,0,12,1422,1148,5130,"Orgain Organic Nutritional Protein Shake, Vani...",Orgain,Health
3,1000001,P00085442,F,0-17,10,A,2,0,12,1057,1595,5130,"FootJoy Womens ASPIRE Golf Shoes (Periwinkle, ...",FootJoy,Sports & Outdoors
4,1000002,P00285442,M,55+,16,C,4+,0,8,7969,217,345,Reebok Mens Club C Revenge Blue Tennis Shoes S...,Reebok,Health


,ProductID,Total_Sales,Total_Reviews,Popularity_Score
2564,2565,1880.0,1,1881.0
1247,1248,1615.0,1,1616.0
739,740,1612.0,1,1613.0
994,995,1562.0,1,1563.0
2750,2751,1470.0,1,1471.0


,Uniq Id,Crawl Timestamp,Product Url,Product Name,Description,List Price,Sale Price,Brand,Item Number,Gtin,Package Size,Category,Postal Code,Available,ProductID,CustID,Label
0,51b010b871cde349bd32159a1cc1a15f,2020-01-24 16:08:36 +0000,https://www.walmart.com/ip/Allegiance-Economy-...,Allegiance Economy Dual-scale Digital Thermometer,We aim to show you accurate product informati...,11.11,11.11,Cardinal Health,NaN,707389636164,NaN,Health,NaN,True,1,1,Rollback
1,d6a7f100e44a626a3701804e99236ad6,2020-01-24 15:54:21 +0000,https://www.walmart.com/ip/Kenneth-Cole-Reacti...,Kenneth Cole Reaction Eau De Parfum Spray For ...,We aim to show you accurate product informati...,23.99,23.99,Kenneth Cole,NaN,191565696101,NaN,Premium Beauty,NaN,True,2,2,Rollback
2,99d2b7da7e3e427a942f864937dacd9d,2020-01-24 18:34:28 +0000,https://www.walmart.com/ip/Kid-Tough-Fitness-I...,Kid Tough Fitness Inflatable Free-Standing Pun...,We aim to show you accurate product informati...,30.76,30.76,BONK FIT,563852139.0,855523007070,NaN,Sports & Outdoors,NaN,True,3,3,Rollback
3,4c76d170c2c6a759cbce812d790a0b88,2020-01-24 11:08:53 +0000,https://www.walmart.com/ip/THE-FIRST-YEARS/167...,THE FIRST YEARS,We aim to show you accurate product informati...,6.99,6.99,The First Years,553299941.0,71463046263,NaN,Baby,NaN,True,4,4,On Sale
4,8ac95837dc8baa01e504fd8f633ffaf2,2020-03-10 07:37:21 +0000,https://www.walmart.com/ip/4-Pack-MD-USA-Seaml...,4 Pack - MD USA Seamless Toe-Wave-In Mesh Diab...,We aim to show you accurate product informatio...,28.27,28.27,MD USA,NaN,191897514500,NaN,Health,NaN,True,5,5,Rollback


Convert CustID and ProductID into a User-Item Matrix.
Train KNN (Item-Based) & SVD (Matrix Factorization).
Generate personalized recommendations for users.

In [38]:
# Check missing values in Brand and Category
print("Missing values in Brand:", df_past_purchases['Brand'].isna().sum())
print("Missing values in Category:", df_past_purchases['Category'].isna().sum())


# Fill missing Brand values with 'Unknown'
df_past_purchases['Brand'].fillna("Unknown", inplace=True)

# Fill missing Category values with 'Other'
df_past_purchases['Category'].fillna("Other", inplace=True)

# Confirm Fix
print("✅ Missing values after cleaning:")
print("Brand:", df_past_purchases['Brand'].isna().sum())
print("Category:", df_past_purchases['Category'].isna().sum())


Missing values in Brand: 0
Missing values in Category: 0
✅ Missing values after cleaning:
Brand: 0
Category: 0


In [10]:
!pip install scikit-surprise


In [48]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate
from sklearn.neighbors import NearestNeighbors

# ✅ Step 1: Validate Past Purchase Data
if df_past_purchases.empty:
    print("❌ df_past_purchases is EMPTY. Check data merging logic.")
else:
    print("✅ df_past_purchases is NOT empty. Proceeding with training...")

# Ensure CustID and ProductID are strings for consistency
df_past_purchases['CustID'] = df_past_purchases['CustID'].astype(str)
df_past_purchases['ProductID'] = df_past_purchases['ProductID'].astype(str)

# ✅ Step 2: Prepare Data for Collaborative Filtering
cf_data = df_past_purchases[['CustID', 'ProductID', 'Purchase']]
reader = Reader(rating_scale=(cf_data['Purchase'].min(), cf_data['Purchase'].max()))
data = Dataset.load_from_df(cf_data, reader)

# ✅ Step 3: Train SVD Model (Collaborative Filtering)
svd_model = SVD()
cross_validate(svd_model, data, cv=5, verbose=True)

# ✅ Step 4: Train KNN Model (Item-Based)
# Ensure no duplicate CustID-ProductID pairs by summing purchases
cf_data_aggregated = cf_data.groupby(['CustID', 'ProductID'])['Purchase'].sum().reset_index()

# Pivot after resolving duplicates
user_item_matrix = cf_data_aggregated.pivot(index='CustID', columns='ProductID', values='Purchase').fillna(0)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
knn_model.fit(user_item_matrix)

# ✅ Step 5: Define Recommendation Function
def recommend_cf(cust_id, num_recommendations=5):
    """ Recommends items using Collaborative Filtering """

    if cust_id not in user_item_matrix.index:
        return "User not found"

    # Get nearest neighbors
    user_vector = user_item_matrix.loc[cust_id].values.reshape(1, -1)
    distances, indices = knn_model.kneighbors(user_vector, n_neighbors=min(num_recommendations + 1, len(user_item_matrix)))

    # Fix: Convert indices to valid column positions
    valid_indices = [i for i in indices[0][1:] if i < len(user_item_matrix.columns)]

    # Fix: Get product IDs safely
    recommended_product_ids = [user_item_matrix.columns[i] for i in valid_indices] if valid_indices else []

    return recommended_product_ids if recommended_product_ids else "No recommendations found"

# ✅ Step 6: Enhance Recommendations with Brand & Category
def recommend_hybrid(cust_id, num_recommendations=5):
    """ Hybrid Recommendations: Combines CF, Brand & Category Similarity """

    # Get CF-based recommendations
    cf_recommendations = recommend_cf(cust_id, num_recommendations)
    if cf_recommendations == "User not found":
        return "User not found"

    # Get past purchases of the customer
    past_purchases = df_past_purchases[df_past_purchases['CustID'] == cust_id]

    # Extract Brands & Categories from past purchases
    preferred_brands = past_purchases['Brand'].unique()
    preferred_categories = past_purchases['Category'].unique()

    # Find products with matching Brand or Category
    brand_category_matches = df_past_purchases[
        (df_past_purchases['Brand'].isin(preferred_brands)) |
        (df_past_purchases['Category'].isin(preferred_categories))
    ]['ProductID'].unique()

    # Merge recommendations
    final_recommendations = list(set(cf_recommendations + list(brand_category_matches)))[:num_recommendations]

    return final_recommendations

# ✅ Step 7: Example Recommendation
if not df_past_purchases.empty:
    test_user = df_past_purchases['CustID'].iloc[0]  # Pick the first available CustID
    print(f"🎯 Hybrid Recommendations for CustID {test_user}: {recommend_hybrid(test_user)}")
else:
    print("❌ No past purchases found. Check data merging logic.")


from surprise.model_selection import cross_validate
from surprise import SVD

# ✅ Train SVD Model Again & Evaluate
svd_model = SVD()
cv_results = cross_validate(svd_model, data, cv=5, verbose=True)

# ✅ Compute Mean RMSE & MAE
mean_rmse = cv_results['test_rmse'].mean()
mean_mae = cv_results['test_mae'].mean()

print("\n📌 Collaborative Filtering (SVD) Model Performance:")
print(f"🔹 Average RMSE: {mean_rmse:.4f}")
print(f"🔹 Average MAE: {mean_mae:.4f}")

✅ df_past_purchases is NOT empty. Proceeding with training...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    15516.581415510.230115498.016715517.290215513.545915511.13297.0181  
MAE (testset)     14683.458714670.700214656.945514680.281414675.598514673.39699.2888  
Fit time          6.05    6.12    6.12    6.19    6.23    6.14    0.06    
Test time         0.58    1.00    0.57    1.01    0.61    0.75    0.20    


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


🎯 Hybrid Recommendations for CustID 5130: ['2054', '928', '2379', '3106', '1607']
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    15505.612715490.777615529.995115496.452815532.786215511.124917.2335 
MAE (testset)     14666.054414651.077414696.289614657.706114695.856814673.396919.1144 
Fit time          6.12    6.31    6.26    6.28    6.21    6.23    0.07    
Test time         1.01    0.58    0.59    1.01    0.59    0.76    0.21    

📌 Collaborative Filtering (SVD) Model Performance:
🔹 Average RMSE: 15511.1249
🔹 Average MAE: 14673.3969


In [31]:
print(df_past_purchases.columns)


Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category',
       'Purchase', 'ProductID', 'CustID', 'Category', 'Product_Name'],
      dtype='object')


Content-Based Filtering Code

In [52]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Prepare Data
df_products_filtered = df_products[['ProductID', 'ProductName', 'Description', 'Category']].fillna('')

# Ensure ProductID is string for consistency
df_products_filtered['ProductID'] = df_products_filtered['ProductID'].astype(str)

# Combine text fields for product similarity
df_products_filtered['combined_text'] = (
    df_products_filtered['ProductName'] + " " +
    df_products_filtered['Description'].replace('', 'No description available') + " " +
    df_products_filtered['Category']
)

# Load a Pretrained Sentence Transformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate Embeddings for Product Text
product_embeddings = np.array(model.encode(df_products_filtered['combined_text'].tolist(), convert_to_numpy=True))

# Step 2: Function to Recommend Similar Products
def recommend_content_based(product_id, num_recommendations=5):
    """ Recommends similar products based on product description similarity """

    if str(product_id) not in df_products_filtered['ProductID'].values:
        return "Product not found"

    # Find index of the given product
    product_index = df_products_filtered.index[df_products_filtered['ProductID'] == str(product_id)].tolist()[0]

    # Compute Similarity Scores
    similarity_scores = cosine_similarity([product_embeddings[product_index]], product_embeddings).flatten()

    # Get Top Similar Products (excluding itself)
    similar_product_indices = np.argsort(similarity_scores)[::-1][1:num_recommendations+1]

    # Return Product IDs and their similarity scores
    recommended_products = df_products_filtered.iloc[similar_product_indices][['ProductID', 'ProductName', 'Category']]
    return recommended_products

# Example Recommendation
test_product = df_products_filtered['ProductID'].iloc[0]  # Get first product as test
recommendations = recommend_content_based(test_product)

print(f"🔍 Content-Based Recommendations for Product {test_product}:")
display(recommendations)


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ✅ Compute Similarity for Content-Based Recommendations
def evaluate_content_based(sample_size=100):
    total_similarity_scores = []

    # Select a random subset of product IDs to evaluate
    sample_products = random.sample(df_past_purchases['ProductID'].unique().tolist(), min(sample_size, len(df_past_purchases['ProductID'].unique())))

    for product_id in sample_products:
        if product_id in df_products_filtered['ProductID'].values:
            recommended_products = recommend_content_based(product_id, num_recommendations=5)

            if not recommended_products.empty:
                recommended_product_ids = recommended_products['ProductID'].tolist()

                product_index = df_products_filtered[df_products_filtered['ProductID'] == product_id].index[0]
                recommended_indices = df_products_filtered[df_products_filtered['ProductID'].isin(recommended_product_ids)].index.tolist()

                if recommended_indices:
                    similarity_scores = cosine_similarity([product_embeddings[product_index]], product_embeddings[recommended_indices]).flatten()
                    total_similarity_scores.extend(similarity_scores)

    avg_similarity = np.mean(total_similarity_scores) if total_similarity_scores else 0
    print(f"\n📌 Content-Based Filtering Model Performance (Sampled {sample_size} Products):")
    print(f"🔹 Average Cosine Similarity: {avg_similarity:.4f}")

# Run Evaluation with a Sample of 100 Products
evaluate_content_based(sample_size=100)


🔍 Content-Based Recommendations for Product 1:


,ProductID,ProductName,Category
14107,14108,Veridian 08-352 Dual Scale 60-Second Digital T...,Health | Medicine Cabinet | Thermometers | Dig...
2864,2865,entrust Performance Digital Thermometers,Health | Medicine Cabinet | Thermometers | Dig...
7923,7924,"Airlife tempo2 thermometer, each part no. 0019...",Health | Medicine Cabinet | Thermometers | Dig...
22364,22365,McKesson Hand-Held Fahrenheit / Celsius Digita...,Health | Medicine Cabinet | Thermometers | Dig...
24617,24618,"Apex Flex Tip Digital Beeper Thermometer, 7006...",Health | Medicine Cabinet | Thermometers | Dig...



📌 Content-Based Filtering Model Performance (Sampled 100 Products):
🔹 Average Cosine Similarity: 0.7073


Hybrid Model (Smart Cart Optimization)
📌 Features of the Hybrid Model
Uses Collaborative Filtering (User-Based)
Recommends items based on past purchase behavior (df_past_purchases).
Incorporates Store-Priority Items (Rollback/Sale)
Prioritizes products labeled "Rollback" or "On Sale" (df_main['Label']).
Fills the Cart to Meet Free Shipping Threshold
Identifies the remaining amount needed for free shipping.
Suggests items to minimize unnecessary spending.

In [53]:
import itertools

def get_hybrid_recommendations(cust_id, current_cart_items, shipping_threshold=20, alpha=0.3):
    """
    Adjusted Hybrid Recommendation System:
    - CF Recommendations (if available).
    - Store-Priority Items sorted by price (Rollback & On Sale).
    - Trending Products for Cold-Start Users.
    """

    # ✅ Step 1: Get Collaborative Filtering Recommendations
    cf_recommendations = recommend_cf(cust_id, num_recommendations=5) if cust_id in df_sales['CustID'].values else []

    # ✅ Step 2: Calculate Current Cart Total
    current_cart_value = df_products[df_products['ProductID'].isin(current_cart_items)]['Price'].sum()

    # ✅ Step 3: Compute Gap to Free Shipping
    gap_to_free_shipping = shipping_threshold - current_cart_value

    # ✅ Step 4: Filter Store-Priority Products ("Rollback" & "On Sale")
    store_priority_items = df_main[df_main['Label'].isin(["Rollback", "On Sale"])].copy()

    # ✅ Step 5: Sort Store-Priority Items by Price (Closest to Filling the Gap)
    store_priority_items = store_priority_items.sort_values(by='Sale Price', ascending=True)

    # ✅ Step 6: Select the Top 5 Store-Priority Products
    store_priority_product_ids = store_priority_items['ProductID'].tolist()[:5]

    # ✅ Step 7: Get Top Trending Products for Cold-Start Users
    trending_products = df_sales['ProductID'].value_counts().index.tolist()[:5]

    # ✅ Step 8: Final Recommendation Logic
    if not cf_recommendations:
        final_recommendations = list(set(store_priority_product_ids + trending_products))[:5]
    else:
        final_recommendations = list(set(cf_recommendations[:int(alpha * 5)] + store_priority_product_ids[:int((1 - alpha) * 5)]))

    return final_recommendations


# 🔥 Example Usage
test_user = df_past_purchases['CustID'].iloc[0]  # Example user
current_cart = [df_products['ProductID'].iloc[0]]  # Example cart
shipping_threshold = 20.0  # Example free shipping limit

print(f"🚀 Hybrid Recommendations for CustID {test_user}: {get_hybrid_recommendations(test_user, current_cart, shipping_threshold)}")

def find_best_gap_filling_combination(gap_filling_pool, gap):
    """
    Finds the best combination of products that fill the remaining amount needed for free shipping.
    """
    best_combination = None
    min_difference = float('inf')

    # Check for an exact match first
    for product in gap_filling_pool:
        item_price = df_products[df_products['ProductID'] == product]['Price'].values[0]
        if abs(item_price - gap) < 1e-6:
            return [product]  # Found an exact match

    # Try different combinations (up to size 3)
    for r in range(1, min(4, len(gap_filling_pool) + 1)):
        for combo in itertools.combinations(gap_filling_pool, r):
            total_price = sum(df_products[df_products['ProductID'] == item]['Price'].values[0] for item in combo)
            difference = abs(total_price - gap)

            if difference < min_difference:
                min_difference = difference
                best_combination = list(combo)

    return best_combination if best_combination else []

# Example Usage
test_user = df_past_purchases['CustID'].iloc[0]  # Example user
current_cart = [df_products['ProductID'].iloc[0]]  # Example cart
shipping_threshold = 20.0  # Example shipping limit

print(f"🚀 Hybrid Recommendations for CustID {test_user}: {get_hybrid_recommendations(test_user, current_cart, shipping_threshold)}")

import itertools

# ✅ Function to Evaluate Gap-Filling Effectiveness
def evaluate_gap_filling():
    gaps_filled = 0
    total_attempts = 0

    for cust_id in df_past_purchases['CustID'].unique():
        current_cart_items = [df_products['ProductID'].iloc[0]]  # Example cart
        shipping_threshold = 20.0  # Example free shipping limit

        recommended_products = get_hybrid_recommendations(cust_id, current_cart_items, shipping_threshold)
        if recommended_products:
            cart_value = df_products[df_products['ProductID'].isin(recommended_products)]['Price'].sum()
            gap = shipping_threshold - cart_value
            if gap <= 0:  # Free shipping achieved
                gaps_filled += 1

        total_attempts += 1

    success_rate = (gaps_filled / total_attempts) * 100 if total_attempts > 0 else 0
    print(f"\n📌 Hybrid Model Performance:")
    print(f"🔹 Percentage of Users Meeting Free Shipping After Recommendations: {success_rate:.2f}%")

# Run Evaluation
evaluate_gap_filling()



🚀 Hybrid Recommendations for CustID 5130: [9664, 6465, 1248, 995, 740]
🚀 Hybrid Recommendations for CustID 5130: [9664, 6465, 1248, 995, 740]

📌 Hybrid Model Performance:
🔹 Percentage of Users Meeting Free Shipping After Recommendations: 100.00%


Evaluating the Model's Performance

In [54]:
import random
import numpy as np

# ✅ Step 1: Filter Users Who Have At Least 2 Past Purchases
valid_users = df_past_purchases.groupby('CustID')['ProductID'].nunique()
valid_users = valid_users[valid_users > 1].index.tolist()

# ✅ Step 2: Sample 15 Users From This Group
sample_users = random.sample(valid_users, min(15, len(valid_users)))

precision_scores = []
recall_scores = []

# ✅ Step 3: Define Precision@K and Recall@K Functions
def precision_at_k(y_true, y_pred, k=5):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred[:k])
    if len(y_pred_set) == 0:
        return 0
    return len(y_true_set & y_pred_set) / len(y_pred_set)

def recall_at_k(y_true, y_pred, k=5):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred[:k])
    if len(y_true_set) == 0:
        return 0
    return len(y_true_set & y_pred_set) / len(y_true_set)

# ✅ Step 4: Evaluate the Hybrid Model for Sampled Users
for cust_id in sample_users:
    actual_purchases = df_past_purchases[df_past_purchases['CustID'] == cust_id]['ProductID'].tolist()

    # Get recommendations
    recommendations = get_hybrid_recommendations(cust_id, [], shipping_threshold=20)

    # Check if recommendations are empty
    if not recommendations:
        print(f"❌ No recommendations found for CustID {cust_id}. Falling back to trending products.")
        recommendations = df_sales['ProductID'].value_counts().index.tolist()[:5]  # Top 5 Trending

    precision = precision_at_k(actual_purchases, recommendations, k=5)
    recall = recall_at_k(actual_purchases, recommendations, k=5)

    precision_scores.append(precision)
    recall_scores.append(recall)

    print(f"✅ CustID {cust_id} | Precision@5: {precision:.4f} | Recall@5: {recall:.4f}")

# ✅ Step 5: Compute Averages
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)

print("\n🔥 Final Model Evaluation (Filtered Sampled 15 Users):")
print(f"🔍 Average Precision@5: {avg_precision:.4f}")
print(f"🔍 Average Recall@5: {avg_recall:.4f}")


✅ CustID 4611 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5504 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 2750 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5005 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 578 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 3240 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 2202 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 4459 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5723 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 4880 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 1626 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 3368 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 2197 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 2606 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 3743 | Precision@5: 0.0000 | Recall@5: 0.0000

🔥 Final Model Evaluation (Filtered Sampled 15 Users):
🔍 Average Precision@5: 0.0000
🔍 Average Recall@5: 0.0000


evaluating models

In [55]:
import random
import numpy as np

# ✅ Step 1: Filter Users Who Have At Least 2 Past Purchases
valid_users = df_past_purchases.groupby('CustID')['ProductID'].nunique()
valid_users = valid_users[valid_users > 1].index.tolist()

# ✅ Step 2: Sample 100 Users From This Group
sample_users = random.sample(valid_users, min(20, len(valid_users)))

precision_scores = []
recall_scores = []

# ✅ Step 3: Define Precision@K and Recall@K Functions
def precision_at_k(y_true, y_pred, k=5):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred[:k])
    return len(y_true_set & y_pred_set) / len(y_pred_set) if y_pred_set else 0

def recall_at_k(y_true, y_pred, k=5):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred[:k])
    return len(y_true_set & y_pred_set) / len(y_true_set) if y_true_set else 0

# ✅ Step 4: Evaluate the Hybrid Model for Sampled Users
for cust_id in sample_users:
    actual_purchases = df_past_purchases[df_past_purchases['CustID'] == cust_id]['ProductID'].tolist()

    # Get recommendations
    recommendations = get_hybrid_recommendations(cust_id, [], shipping_threshold=20)

    # Check if recommendations are empty
    if not recommendations:
        print(f"❌ No recommendations found for CustID {cust_id}. Falling back to trending products.")
        recommendations = df_sales['ProductID'].value_counts().index.tolist()[:5]  # Top 5 Trending

    precision = precision_at_k(actual_purchases, recommendations, k=5)
    recall = recall_at_k(actual_purchases, recommendations, k=5)

    precision_scores.append(precision)
    recall_scores.append(recall)

    print(f"✅ CustID {cust_id} | Precision@5: {precision:.4f} | Recall@5: {recall:.4f}")

# ✅ Step 5: Compute Averages
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)

print("\n🔥 Final Model Evaluation (Filtered Sampled 100 Users):")
print(f"🔍 Average Precision@5: {avg_precision:.4f}")
print(f"🔍 Average Recall@5: {avg_recall:.4f}")


✅ CustID 5728 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5226 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 3226 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 310 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 3982 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 2402 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 4948 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 970 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5065 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 1726 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 935 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 3836 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 4305 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 4344 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 4870 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 494 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5278 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 5154 | Precision@5: 0.0000 | Recall@5: 0.0000
✅ CustID 497 |

**Building the LLM**

In [7]:
print(df_main.columns.tolist())
print(df_products.columns.tolist())
print(df_sales.columns.tolist())
print(df_reviews.columns.tolist())

['Uniq Id', 'Crawl Timestamp', 'Product Url', 'Product Name', 'Description', 'List Price', 'Sale Price', 'Brand', 'Item Number', 'Gtin', 'Package Size', 'Category', 'Postal Code', 'Available', 'ProductID', 'CustID', 'Label']
['Unnamed: 0', 'ProductID', 'ProductName', 'Description', 'Price', 'Category', 'StockDate']
['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category', 'Purchase', 'ProductID', 'CustID', 'Category', 'Product_Name']
['customer_id', 'product_id', 'uniq_id', 'review', 'label', 'name', 'location', 'Date', 'Rating', 'Name', 'Location', 'Image_Links', 'CustID', 'ProductID']


In [8]:
import pandas as pd
import numpy as np
import random
from flask import Flask, request, jsonify
from transformers import pipeline
from flask_ngrok import run_with_ngrok

# ✅ Step 1: Standardize Column Names
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

df_main = clean_columns(df_main)
df_products = clean_columns(df_products)
df_sales = clean_columns(df_sales)
df_reviews = clean_columns(df_reviews)

print("df_main columns:", df_main.columns.tolist())
print("df_products columns:", df_products.columns.tolist())
print("df_sales columns:", df_sales.columns.tolist())
print("df_reviews columns:", df_reviews.columns.tolist())

# ✅ Step 2: Select Important Fields
important_fields = [
    'productid', 'product_name', 'brand', 'category', 'price', 'sale_price',  # Product details
    'available', 'label', 'stock_date',  # Availability & Promotions
    'total_sales', 'avg_rating', 'economic_factor'  # Extra features for ML
]

# Ensure selection only includes available columns
selected_cols = [col for col in important_fields if col in df_products.columns]
df_products_final = df_products[selected_cols].copy()

# ✅ Step 3: Merge Availability & Rollback Labels from `df_main`
if {'productid', 'available', 'label'}.issubset(df_main.columns):
    df_products_final = df_products_final.merge(df_main[['productid', 'available', 'label']], on='productid', how='left')

# ✅ Step 4: Merge Sales Data from `df_sales` (Using `CustID` and `ProductID`)
if {'custid', 'productid', 'purchase'}.issubset(df_sales.columns):
    df_sales_summary = df_sales.groupby('productid')['purchase'].sum().reset_index()
    df_sales_summary.rename(columns={'purchase': 'total_sales'}, inplace=True)
    df_products_final = df_products_final.merge(df_sales_summary, on='productid', how='left')

# ✅ Step 5: Merge Customer Reviews from `df_reviews` (Using `CustID` and `ProductID`)
df_reviews.rename(columns={'Rating': 'rating'}, inplace=True)

if {'custid', 'productid', 'rating'}.issubset(df_reviews.columns):
    df_reviews_summary = df_reviews.groupby('productid').agg({'rating': 'mean'}).reset_index()
    df_reviews_summary.rename(columns={'rating': 'avg_rating'}, inplace=True)
    df_products_final = df_products_final.merge(df_reviews_summary, on='productid', how='left')

# ✅ Step 6: Simulate Economic Factors
economic_factors = ['Inflation', 'Unemployment Rate', 'GDP Growth']
df_products_final['economic_factor'] = np.random.choice(economic_factors, size=len(df_products_final))

# ✅ Step 7: Fill Missing Values
df_products_final.fillna({
    'available': 'Unknown', 'label': 'None', 'total_sales': 0, 'avg_rating': 0, 'economic_factor': 'Stable'
}, inplace=True)

# ✅ Step 8: Validate Final Data
print("✅ Data Standardization Complete!")
print("df_products_final columns:", df_products_final.columns.tolist())
display(df_products_final.head())

# ✅ Step 9: Load a Pre-Trained LLM (GPT-Neo)
llm = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

def recommend_products(query):
    prompt = f"""
    You are an AI assistant recommending products based on the following query:
    Query: "{query}"

    Consider the following factors:
    - **Availability** (available products are prioritized)
    - **Price** (recommend cost-effective options)
    - **High Sales Volume** (popular products)
    - **Rollback/Sale Labels** (products currently on discount)
    - **Economic Conditions** (inflation, GDP trends)

    Product Database:
    {df_products_final[['product_name', 'category', 'price', 'label', 'total_sales', 'available', 'economic_factor']].head(10).to_string()}

    Give a clear and detailed recommendation.
    """

    response = llm(prompt, max_length=150, num_return_sequences=1)
    return response[0]['generated_text']

# ✅ Step 10: Create Flask API
app = Flask(__name__)
run_with_ngrok(app)  # Allow public access on Colab

@app.route('/recommend', methods=['GET'])
def recommend():
    user_query = request.args.get('query', default="Suggest some products", type=str)
    response = recommend_products(user_query)
    return jsonify({"query": user_query, "recommendation": response})

if __name__ == '__main__':
    print("✅ Starting Flask Server...")
    app.run()


df_main columns: ['uniq_id', 'crawl_timestamp', 'product_url', 'product_name', 'description', 'list_price', 'sale_price', 'brand', 'item_number', 'gtin', 'package_size', 'category', 'postal_code', 'available', 'productid', 'custid', 'label']
df_products columns: ['unnamed:_0', 'productid', 'productname', 'description', 'price', 'category', 'stockdate']
df_sales columns: ['user_id', 'product_id', 'gender', 'age', 'occupation', 'city_category', 'stay_in_current_city_years', 'marital_status', 'product_category', 'purchase', 'productid', 'custid', 'category', 'product_name']
df_reviews columns: ['customer_id', 'product_id', 'uniq_id', 'review', 'label', 'name', 'location', 'date', 'rating', 'name', 'location', 'image_links', 'custid', 'productid']
✅ Data Standardization Complete!
df_products_final columns: ['productid', 'category', 'price', 'available', 'label', 'total_sales', 'avg_rating', 'economic_factor']


,productid,category,price,available,label,total_sales,avg_rating,economic_factor
0,1,Health | Medicine Cabinet | Thermometers | Dig...,11.11,True,Rollback,656994.0,4.0,Inflation
1,2,Premium Beauty | Premium Fragrance | Premium P...,23.99,True,Rollback,336105.0,2.0,Inflation
2,3,Sports & Outdoors | Outdoor Sports | Hunting |...,30.76,True,Rollback,1002743.0,2.0,Inflation
3,4,Baby | Diapering | Baby Wipes,6.99,True,On Sale,9054374.0,1.0,GDP Growth
4,5,Health | Diabetes Care | Diabetic Socks,28.27,True,Rollback,227908.0,1.0,Unemployment Rate


Device set to use cuda:0


✅ Starting Flask Server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-22:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# ✅ Function to Standardize Column Names
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower()  # Normalize column names
    return df

# ✅ Apply Standardization BEFORE merging
df_main = clean_columns(df_main)
df_products = clean_columns(df_products)
df_sales = clean_columns(df_sales)
df_reviews = clean_columns(df_reviews)

# ✅ Identify Duplicate Columns & Remove Them
def check_duplicate_columns(df, df_name):
    duplicate_cols = df.columns[df.columns.duplicated()]
    if not duplicate_cols.empty:
        print(f"⚠️ Duplicate columns found in {df_name}: {duplicate_cols.tolist()}")
        df = df.loc[:, ~df.columns.duplicated()]  # Keep only unique columns
        print(f"✅ After cleanup, {df_name} columns: {df.columns.tolist()}")
    return df

df_main = check_duplicate_columns(df_main, "df_main")
df_products = check_duplicate_columns(df_products, "df_products")
df_sales = check_duplicate_columns(df_sales, "df_sales")
df_reviews = check_duplicate_columns(df_reviews, "df_reviews")

# ✅ Aggregate `df_sales` If Customers Have Multiple Purchases
print("🔍 Checking for duplicate `custid` purchases in `df_sales`...")

agg_dict = {}
if 'purchase' in df_sales.columns:
    agg_dict['purchase'] = 'sum'  # Sum up purchase count
if 'price' in df_sales.columns:
    agg_dict['price'] = 'mean'  # Average price paid
if 'label' in df_sales.columns:
    agg_dict['label'] = 'first'  # Keep the first label (Rollback, Sale, etc.)

# ✅ Aggregate purchases by `custid` and `productid`
if agg_dict:
    df_sales = df_sales.groupby(['custid', 'productid']).agg(agg_dict).reset_index()
    print("✅ Aggregated df_sales:", df_sales.head())
else:
    print("⚠️ No valid columns found for aggregation. Skipping this step.")

# ✅ Remove Duplicate `productid` Entries in `df_products`
if 'productid' in df_products.columns:
    df_products = df_products.drop_duplicates(subset=['productid'], keep="first")

# ✅ Merge All Data into a Single DataFrame
df = df_sales.merge(df_products[['productid', 'price', 'category']], on="productid", how="left")

if "productid" in df.columns and "productid" in df_reviews.columns:
    df = df.merge(df_reviews[['productid', 'rating']], on="productid", how="left")

if "custid" in df.columns and "custid" in df_main.columns:
    df = df.merge(df_main[['custid', 'productid', 'brand', 'label']], on=["custid", "productid"], how="left")

# ✅ Fix `custid_x` and `custid_y` Issues (Drop Unnecessary Duplicates)
df = df.rename(columns={'custid_x': 'custid'}).drop(columns=['custid_y'], errors='ignore')

# ✅ Ensure 'brand' Exists in Final Dataset
if 'brand' not in df.columns:
    df['brand'] = 'Unknown'

# ✅ Handle Missing Values
df.fillna(0, inplace=True)

# ✅ Print Final Columns
print("✅ Final Merged Dataset Columns:", df.columns.tolist())

# ✅ Select Features & Labels for ML
features = df[['custid', 'productid', 'category', 'price', 'brand', 'label', 'rating']]
labels = df['purchase']  # Target: Whether the user purchased or not

# ✅ Split Data into Train & Test
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# ✅ Display Shapes for Validation
print(f"✅ Training Set: {X_train.shape}, {y_train.shape}")
print(f"✅ Test Set: {X_test.shape}, {y_test.shape}")


⚠️ Duplicate columns found in df_reviews: ['name', 'location']
✅ After cleanup, df_reviews columns: ['customer_id', 'product_id', 'uniq_id', 'review', 'label', 'name', 'location', 'date', 'rating', 'image_links', 'custid', 'productid']
🔍 Checking for duplicate `custid` purchases in `df_sales`...
✅ Aggregated df_sales:    custid  productid  purchase
0       1         22     13421
1       1         81      3186
2       1         85      6868
3       1        254     15318
4       1        406     19544
✅ Final Merged Dataset Columns: ['custid', 'productid', 'purchase', 'price', 'category', 'rating', 'brand', 'label']
✅ Training Set: (440054, 7), (440054,)
✅ Test Set: (110014, 7), (110014,)


In [23]:
from google.colab import files

# Define the file path
file_path = "final_merged_dataset.csv"

# Save the DataFrame to CSV
df.to_csv(file_path, index=False)

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ✅ Load Data (Assuming `df` is already prepared)
df = df.copy()  # Ensure original dataset is not modified

# ✅ Convert categorical variables to string type to avoid type errors
categorical_cols = ['category', 'brand', 'label']
df[categorical_cols] = df[categorical_cols].astype(str)

# ✅ Encode Categorical Variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders in case we need them later

# ✅ Ensure numerical features are of float type
df[['price', 'rating']] = df[['price', 'rating']].astype(float)

# ✅ Standardize Numerical Features
scaler = StandardScaler()
df[['price', 'rating']] = scaler.fit_transform(df[['price', 'rating']])

# ✅ Split Data into Train & Test Sets
features = df[['custid', 'productid', 'category', 'price', 'brand', 'label', 'rating']]
labels = df['purchase']
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# ✅ Define ML Models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    # "Neural Network (MLP)": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500)
}

# ✅ Train & Evaluate Models
results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results[model_name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, zero_division=1),
        "Recall": recall_score(y_test, y_pred, zero_division=1),
        "F1 Score": f1_score(y_test, y_pred, zero_division=1)
    }

# ✅ Display Results
results_df = pd.DataFrame(results).T
print("\n✅ Model Performance:\n")
print(results_df)

# ✅ Select Best Model Based on F1-Score
best_model_name = results_df['F1 Score'].idxmax()
print(f"\n🏆 Best Model Selected: {best_model_name}")


NameError: name 'df' is not defined

In [12]:
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# ✅ Load Data (Ensure df is already prepared)
df = df.copy()  # Ensure original dataset is not modified

# ✅ Convert Columns to Proper Data Types
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# ✅ Encode Categorical Variables
label_encoders = {}
categorical_cols = ['category', 'brand', 'label']

for col in categorical_cols:
    df[col] = df[col].astype(str)  # Convert all categorical columns to string
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for future use

# ✅ Standardize Numerical Features
scaler = StandardScaler()
df[['price', 'rating']] = scaler.fit_transform(df[['price', 'rating']])

# ✅ Convert Target to Binary Classification
df['purchase'] = (df['purchase'] > 0).astype(int)  # Convert to binary (1 = bought, 0 = not bought)

# ✅ Split Data into Train & Test Sets
features = df[['custid', 'productid', 'category', 'price', 'brand', 'label', 'rating']]
labels = df['purchase']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# ✅ Define ML Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    "Neural Network (MLP)": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500)
}

# ✅ Train & Evaluate Models (One at a Time to Prevent Crashes)
results = {}

for model_name, model in models.items():
    print(f"\n🚀 Training {model_name} (Clearing RAM)...")

    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Predict on test set

    # ✅ Store Performance Metrics
    results[model_name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred)
    }

    # ✅ Save Model to Disk to Free Memory
    joblib.dump(model, f"{model_name.replace(' ', '_')}.joblib")
    print(f"✅ {model_name} saved to disk.")

    # ✅ Free Up Memory Before Next Training
    del model, y_pred
    gc.collect()  # Run garbage collector

# ✅ Display Results
results_df = pd.DataFrame(results).T
print("\n📊 Model Performance:\n")
print(results_df)

# ✅ Select Best Model Based on F1 Score
best_model_name = results_df['F1 Score'].idxmax()
print(f"\n🏆 Best Model Selected: {best_model_name}")



🚀 Training Logistic Regression (Clearing RAM)...


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

➡️ Create User-Item Matrix

In [13]:
import gradio as gr
from transformers import pipeline

# ✅ Load the LLM Model
llm = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

# ✅ Define the Product Recommendation Function
import pandas as pd

# ✅ Load Your Product Dataset
df_products = df_products_final  # Replace with your actual dataset

def recommend_products(query):
    # Filter products that match the query (e.g., shoes on sale)
    filtered_products = df_products[
        df_products["category"].str.contains("shoes", case=False, na=False) &
        df_products["label"].str.contains("sale|rollback", case=False, na=False)
    ]

    if not filtered_products.empty:
        top_product = filtered_products.iloc[0]
        return f"🔥 Recommended Product: {top_product['product_name']} - ${top_product['price']}\n Available: {top_product['available']}"
    else:
        return "❌ No matching products found."

# ✅ Create the Gradio UI



demo = gr.Interface(
    fn=recommend_products,
    inputs=gr.Textbox(label="Enter your query"),
    outputs=gr.Textbox(label="Recommended Products"),
    title="🛍 AI-Powered Product Recommendation System",
    description="Find discounted or high-selling products based on your needs."
)

demo.launch(share=True)


# ✅ Launch Gradio in Colab
demo.launch(share=True)


ModuleNotFoundError: No module named 'gradio'